# Data:

In [1]:
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
# Indicate dataframes to import.
list_dfs = ['pickled_conala_mined_df', 'pickled_conala_train_df', 'pickled_conala_test_df',
           'conala_train_bag_df', 'conala_mined_bag_df', 'combined_bag_df']

In [3]:
%time
# Load all data in list_dfs
data = {}
for df in list_dfs:
    dbfile = open(df, 'rb')      
    contents = pickle.load(dbfile)
    data[df] = contents
    dbfile.close()

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.05 µs


In [4]:
data.keys()

dict_keys(['pickled_conala_mined_df', 'pickled_conala_train_df', 'pickled_conala_test_df', 'conala_train_bag_df', 'conala_mined_bag_df', 'combined_bag_df'])

In [5]:
df = data['combined_bag_df']

## Word2Vec

For Word2Vec, we need a list of all the sentences which will be transformed in it. So this will have to be done for both intent, and snippet. We can assemble this by combining the `conala_train_df` and the `conala_mined_df`

In [6]:
conala_train_df = data["pickled_conala_train_df"]
conala_mined_df = data["pickled_conala_mined_df"]

In [7]:
# concatenate the two dfs.
df = pd.concat([conala_train_df, conala_mined_df], ignore_index=True)

In [8]:
# Peek
df

,intent,rewritten_intent,snippet,question_id,parent_answer_post_id,prob,id
0,How to convert a list of multiple integers int...,Concatenate elements of a list 'x' of multiple...,"sum(d * 10 ** i for i, d in enumerate(x[::-1]))",41067960,NaN,NaN,NaN
1,How to convert a list of multiple integers int...,convert a list of integers into a single integer,"r = int(''.join(map(str, x)))",41067960,NaN,NaN,NaN
2,how to convert a datetime string back to datet...,convert a DateTime string back to a DateTime o...,datetime.strptime('2010-11-13 10:33:54.227806'...,4170655,NaN,NaN,NaN
3,Averaging the values in a dictionary based on ...,get the average of a list values for each key ...,"[(i, sum(j) / len(j)) for i, j in list(d.items...",29565452,NaN,NaN,NaN
4,zip lists in python,"zip two lists `[1, 2]` and `[3, 4]` into a lis...","zip([1, 2], [3, 4])",13704860,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5759,How to convert datetime to string in python in...,NaN,{{(item.date | date): 'Y M d'}},794995,795000.0,0.500243,794995_795000_0
5760,Delete column from pandas DataFrame,NaN,"df = df.drop('column_name', 1)",13411544,18145399.0,0.500193,13411544_18145399_2
5761,How to get a list which is a value of a dictio...,NaN,"reverse_d = {value: key for key, values in lis...",40584186,40584271.0,0.500171,40584186_40584271_0
5762,Cross-platform addressing of the config file,NaN,config_file = os.path.expanduser('~/foo.ini'),3227624,3227931.0,0.500164,3227624_3227931_0


In [9]:
# Create a list of the text in intent field. (Note this is NOT using the 
# rewritten intent in the training data.)
intent_text = list(df["intent"])

# Create a list of the code snippets in the data. 
snippet_text = list(df["snippet"])

In [31]:
intent_corpus = conala_train_df["rewritten_intent"].str.cat(sep=', ')

In [32]:
intent_corpus

'Concatenate elements of a list \'x\' of multiple integers to a single integer, convert a list of integers into a single integer, convert a DateTime string back to a DateTime object of format \'%Y-%m-%d %H:%M:%S.%f\', get the average of a list values for each key in dictionary `d`), zip two lists `[1, 2]` and `[3, 4]` into a list of two tuples containing elements at the same index in each list, prepend string \'hello\' to all items in list \'a\', regex for repeating words in a string `s`, normalize a pandas dataframe `df` by row, swap values in a tuple/list inside a list `mylist`, Swap values in a tuple/list in list `mylist`, find all occurrences of the pattern \'\\\\[[^\\\\]]*\\\\]|\\\\([^\\\\)]*\\\\)|"[^"]*"|\\\\S+\' within `strs`, generate the combinations of 3 from a set `{1, 2, 3, 4}`, add multiple columns `hour`, `weekday`, `weeknum` to pandas data frame `df` from lambda function `lambdafunc`, BeautifulSoup search string \'Elsie\' inside tag \'a\', Convert a datetime object `my_d

In [33]:
import re
import nltk
nltk.download('punkt')
# Cleaning the text
processed_intent = intent_corpus.lower()
processed_intent = re.sub('[^a-zA-Z]', ' ', processed_intent)
processed_intent = re.sub(r'\s+', ' ', processed_intent)

# Preparing the dataset
all_sentences = nltk.sent_tokenize(processed_intent)

all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

# Removing Stop Words
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/justin.hugh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_words, min_count=10)


In [49]:
v1 = word2vec.wv['without']

In [50]:
v1

array([-0.23350613, -0.06681756, -0.09248789, -0.06195335,  0.17670427,
        0.1273676 ,  0.04220295,  0.05662565, -0.00152902, -0.10956798,
       -0.0059781 , -0.11800621, -0.17211758, -0.03234676, -0.1987067 ,
       -0.08451557,  0.09210501, -0.05223181,  0.136691  , -0.09519101,
        0.03352926,  0.06373753, -0.14599724, -0.10923551,  0.06430598,
        0.08535234, -0.14117499,  0.05025436,  0.26767084,  0.23914137,
        0.15752031,  0.1956209 , -0.13645898, -0.04619893, -0.06376147,
        0.13758336, -0.23159675,  0.08978047, -0.07612997, -0.04746798,
       -0.0383179 , -0.02319166, -0.01582518, -0.11564402,  0.06209053,
       -0.15332077, -0.00260589,  0.14263602,  0.1303455 , -0.24938013,
       -0.22792254, -0.04014506,  0.10579573,  0.04075888, -0.05405435,
        0.07344197, -0.01297047, -0.16357431,  0.18630847,  0.04174403,
       -0.21746013, -0.12386032,  0.04006714,  0.17915547,  0.02205357,
        0.06002088,  0.07811099,  0.010624  , -0.01383098,  0.03

In [48]:
sim_words = word2vec.wv.most_similar('without')

In [45]:
sim_words

[('integers', 0.9994053840637207),
 ('second', 0.9993956089019775),
 ('using', 0.9993821978569031),
 ('list', 0.9993412494659424),
 ('python', 0.9993401765823364),
 ('format', 0.9993376135826111),
 ('index', 0.9993231296539307),
 ('dictionary', 0.9993177056312561),
 ('foo', 0.9993091821670532),
 ('month', 0.9993070960044861)]

In [21]:
v1.shape

(100,)

In [47]:
vocabulary = word2vec.wv.vocab
print(vocabulary)

{'concatenate': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3b3d0>, 'elements': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3b350>, 'list': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3b310>, 'x': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3b290>, 'multiple': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3b050>, 'integers': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3ae10>, 'single': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3a9d0>, 'integer': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3a910>, 'convert': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3b0d0>, 'datetime': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3af50>, 'string': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3a610>, 'object': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3a550>, 'format': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3a4d0>, 'h': <gensim.models.keyedvectors.Vocab object at 0x7fd1d0f3a450>, 'f

In [10]:
# Check
print(intent_text[:10])
print(snippet_text[:10])

['How to convert a list of multiple integers into a single integer?', 'How to convert a list of multiple integers into a single integer?', 'how to convert a datetime string back to datetime object?', 'Averaging the values in a dictionary based on the key', 'zip lists in python', 'Prepend the same string to all items in a list', 'regex for repeating words in a string in Python', 'Normalizing a pandas DataFrame by row', 'swap values in a tuple/list inside a list in python?', 'swap values in a tuple/list inside a list in python?']
['sum(d * 10 ** i for i, d in enumerate(x[::-1]))', "r = int(''.join(map(str, x)))", "datetime.strptime('2010-11-13 10:33:54.227806', '%Y-%m-%d %H:%M:%S.%f')", '[(i, sum(j) / len(j)) for i, j in list(d.items())]', 'zip([1, 2], [3, 4])', "['hello{0}'.format(i) for i in a]", "re.sub('(?<!\\\\S)((\\\\S+)(?:\\\\s+\\\\2))(?:\\\\s+\\\\2)+(?!\\\\S)', '\\\\1', s)", 'df.div(df.sum(axis=1), axis=0)', 'map(lambda t: (t[1], t[0]), mylist)', '[(t[1], t[0]) for t in mylist]']

Now we need to get each unique word in the text, and for the code, each unique char.

In [11]:
# Get unique words in text
intent_tokens = set()
    
for intent in tqdm(intent_text):
    for word in intent.split(" "):
        intent_tokens.add(word)

num_intent_tokens = len(intent_tokens)
intent_tokens

100%|██████████| 5764/5764 [00:00<00:00, 204870.67it/s]


{'float',
 'compose',
 'possible',
 'entry',
 '',
 'Horizontal',
 'Iterate',
 'Bulk',
 'fixed',
 'only?',
 'variables',
 'capital',
 'dot',
 'app?',
 'gradients',
 '0s',
 'Max',
 'box?',
 'tabs',
 'Pillow',
 'converting',
 'cookie',
 'based',
 're-indexing',
 'utf-string',
 'Office',
 'Time',
 'Expat',
 'xlwt?',
 'go',
 'Evaluating',
 'listing',
 'pylab.savefig()',
 'str(dict)?',
 'figure?',
 'psycopg2:',
 '0-dimension',
 'gracefully',
 'y-axis',
 'concise',
 'Contained',
 'place',
 'index?',
 'parse',
 'configure',
 'App',
 'modules',
 'sub-level',
 'python,',
 'libraries?',
 'unicode_literals',
 'fashion',
 'occurences',
 'comma-separated',
 '(float)',
 'correspond',
 'value)',
 'groupby:',
 'Flask-Mail',
 'them',
 'doubling',
 'integrate',
 'bower',
 'year?',
 'pipe?',
 'interactive?',
 'Uniqueness',
 'id?',
 'crop',
 'sets',
 'pandas',
 'driver',
 "'£'",
 'SQLAlchemy-flask',
 'Shell',
 'Encoding',
 'Pyhon',
 'extension?',
 'INSERT',
 'visible?',
 'repeating',
 'slow',
 'installed',

In [12]:
len(intent_text)

5764

In [13]:
num_intent_tokens

3658

In [14]:
# Create the data with N-grams
from nltk import ngrams
import itertools

gram_size = 4
data = []

# Go over each intent statement
for intent in tqdm(intent_text):
    # Finds all n-grams in the statement
    grams = ngrams(intent.split(), gram_size)
    for gram in grams:
        # Find all pairs of words within this n-gram
        for pair in itertools.permutations(gram, 2):
            data.append(pair)

data[0:20]

100%|██████████| 5764/5764 [00:00<00:00, 27912.57it/s]


[('How', 'to'),
 ('How', 'convert'),
 ('How', 'a'),
 ('to', 'How'),
 ('to', 'convert'),
 ('to', 'a'),
 ('convert', 'How'),
 ('convert', 'to'),
 ('convert', 'a'),
 ('a', 'How'),
 ('a', 'to'),
 ('a', 'convert'),
 ('to', 'convert'),
 ('to', 'a'),
 ('to', 'list'),
 ('convert', 'to'),
 ('convert', 'a'),
 ('convert', 'list'),
 ('a', 'to'),
 ('a', 'convert')]

In [15]:
len(data)

400836

In [29]:
from sklearn.preprocessing import LabelBinarizer
from scipy import sparse
from scipy.sparse import csr_matrix

In [54]:
encoder = LabelBinarizer(sparse_output=False)

In [55]:
one_hot_encoder = encoder.fit(list(intent_tokens))

In [56]:
len(one_hot_encoder.classes_)

3658

In [57]:
#Transform the input/output pairs:
intent_train_data = []
intent_train_target = []

for pair in tqdm(data[:1000]):
    intent_train_data.append(one_hot_encoder.transform([pair[0]]))
    intent_train_target.append(one_hot_encoder.transform([pair[1]]))

100%|██████████| 1000/1000 [00:04<00:00, 217.55it/s]


In [58]:
intent_train_data = np.squeeze(np.asarray(intent_train_data))
intent_train_target = np.squeeze(np.asarray(intent_train_target))

In [37]:
%time
# Pickle the data for use later, avoiding lengthy one-hot encoding again. 
# intent_train_data
with open('pickled_intent_train_data.pkl', 'wb+') as f:
    # source, destination 
    pickle.dump(intent_train_data, f)                      

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [60]:
intent_train_data.shape

(1000, 3658)

In [62]:
# Cleaning the text
processed_intent_text = intent_text.lower()

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
processed_article = re.sub(r'\s+', ' ', processed_article)

# Preparing the dataset
all_sentences = nltk.sent_tokenize(processed_article)

all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

# Removing Stop Words
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

In [59]:
from gensim.models import Word2Vec
word2vec = Word2Vec(intent_train_data, min_count=5)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U21'), dtype('<U21')) -> dtype('<U21')

In [50]:
vocabulary = word2vec.wv.vocab

In [38]:
# intent_train_target
with open('pickled_intent_train_target.pkl', 'wb+') as f:
    # source, destination 
    pickle.dump(intent_train_target, f)                      

Now set up the network. 

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
model = keras.models.Sequential()

model.add(keras.layers.Dense(10, activation='relu'))

# Output Layer
model.add(keras.layers.Dense(num_intent_tokens, activation='softmax'))

model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.CategoricalCrossentropy()
)

In [25]:
num_epochs = 1000

# Printout a single verbose fit operation 10 times throughout the training process.
for i in range(0, 10):
    model.fit(intent_train_data, intent_train_target, epochs=round(num_epochs/10)-1, verbose=0)
    
    print(f"Epoch: {(i+1)*round(num_epochs/10)}/{num_epochs}")
    model.fit(intent_train_data, intent_train_target, verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_matrix).